In [1]:
import cv2
import numpy as np
img = cv2.imread('numbers.png');

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(5,5),0)
thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

samples = np.empty((0,100))
response = []
keys = [i for i in range(48,58)]

In [2]:
for cnt in contours:
    if cv2.contourArea(cnt)>50:
        [x,y,w,h] = cv2.boundingRect(cnt)
        
        if h>28:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 2)
            roi = thresh[y:y+h, x:x+w]
            roismall = cv2.resize(roi,(10,10))
            cv2.imshow('Original', img)
            key = cv2.waitKey(0)
            
            if key == 27:  
                print("TODO")
            elif key in keys:
                response.append(int(chr(key)))
                sample = roismall.reshape((1,100))
                samples = np.append(samples,sample,0)
                                
response = np.array(response, np.float32)
response = response.reshape((response.size, 1))
                                
np.savetxt("samples.data", samples)                            
np.savetxt("response.data", response)

print("Cascade great")

Cascade great


In [ ]:
import cv2
import numpy as np
img = cv2.imread('5028.png');

samples = np.loadtxt("samples.data", np.float32)
response = np.loadtxt("response.data", np.float32)

response = response.reshape((response.size, 1))

model = cv2.ml.KNearest_create()
model.train(samples, cv2.ml.ROW_SAMPLE, response)

out = np.zeros(img.shape, np.uint8)

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    if cv2.contourArea(cnt)>50:
        [x,y,w,h] = cv2.boundingRect(cnt)
        
        if h>28:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 2)
            roi = thresh[y:y+h, x:x+w]
            roismall = cv2.resize(roi,(10,10))
            roismall = roismall.reshape((1,100))
            roismall = np.float32(roismall)
            retval, result, neight_resp, dist = model.findNearest(roismall, k=1)
            string = str(int(result[0][0]))
            print(result[0][0])
            cv2.putText(out,string, (x, y+h), 0, 1, (0,255,0))
            
            
cv2.imshow('Original', img)
cv2.imshow('Result', out)
cv2.waitKey(0)

2.0
2.0
6.0
8.0
2.0
0.0
5.0
4.0
